# **<font color=blue> PROYECTO FINAL**
## <font color=blue> RED NEURONAL RECURRENTE

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import re

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dropout
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import BatchNormalization

In [3]:
Train = pd.read_csv('./train.csv', names = ['sentiment', 'text'] )
Test  = pd.read_csv('./test.csv',  names = ['sentiment', 'text'] )
Train.head()

,sentiment,text
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [4]:
Train =Train.loc[:6000,:]
Test = Test.loc[:2000,:]
Train.isnull().sum()

sentiment    0
text         0
dtype: int64

In [5]:
Train['sentiment'].value_counts()

1    3102
2    2899
Name: sentiment, dtype: int64

In [6]:
Train['text'] = Train['text'].apply(lambda x: x.lower())
Train['text'] = Train['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
Test['text'] = Test['text'].apply(lambda x: x.lower())
Test['text'] = Test['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [7]:
token = Tokenizer(num_words=1000, split=' ')
token.fit_on_texts(Train['text'].values)
X = token.texts_to_sequences(Train['text'].values)
X = pad_sequences(X)

In [8]:
token.fit_on_texts(Test['text'].values)
X_Val = token.texts_to_sequences(Test['text'].values)
X_Val = pad_sequences(X_Val)

In [9]:
X.shape

(6001, 807)

In [10]:
Y = pd.get_dummies(Train['sentiment']).values
Y_Val = pd.get_dummies(Test['sentiment']).values

In [11]:
Y.shape

(6001, 2)

In [12]:
#DEFINICION DE MODELO
model = Sequential()
model.add(Embedding(1000, 128,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 807, 128)          128000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 807, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 383,194
Trainable params: 383,194
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
model.fit(X, Y, epochs = 2, batch_size=50, verbose = 1)

Epoch 1/2
121/121 [==============================] - 741s 6s/step - loss: 0.5383 - accuracy: 0.7225
Epoch 2/2
121/121 [==============================] - 736s 6s/step - loss: 0.3491 - accuracy: 0.8514


In [27]:
score,acc = model.evaluate(X_Val, Y_Val, verbose = 1, batch_size = 50)
MLP_K = pd.DataFrame({
    'Recurrente': ['Keras'],
    'Acurracy': [acc]
})
MLP_K

41/41 [==============================] - 28s 687ms/step - loss: 1.2422 - accuracy: 0.4963


,Recurrente,Acurracy
0,Keras,0.496252
